# An introduction to `relatio` 
**Runtime $\sim$ 5min**

Original paper: ["Text Semantics Capture Political and Economic Narratives"](https://arxiv.org/abs/2108.01720)

----------------------------

This is a short demo of the package `relatio`.  It takes as input a text corpus and outputs a list of narrative statements. The pipeline is unsupervised: the user does not need to specify narratives beforehand. Narrative statements are defined as tuples of semantic roles with a (agent, verb, patient, attribute) structure. 

Here, we present the main wrapper functions to quickly obtain narrative statements from a corpus.

----------------------------

In this tutorial, we work with tweets from candidates at the French Presidential Elections (2022).

----------------------------

In [1]:
# Catch warnings for an easy ride
from relatio import FileLogger
logger = FileLogger(level = 'WARNING')

/home/germain/Desktop/relatio/.tox/dev/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/germain/Desktop/relatio/.tox/dev/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
from relatio import list_data
list_data()


{
    "trump_tweet_archive": 
    {
        "description": "Tweets from the Trump Tweet Archives (https://www.thetrumparchive.com/)",
        "language": "english",
        "srl_model": "allennlp v0.9 -- srl-model-2018.05.25.tar.gz",
        "links": 
        {
            "raw": "https://www.dropbox.com/s/lxqz454n29iqktn/trump_archive.csv?dl=1",
            "sentences": "https://www.dropbox.com/s/coh4ergyrjeolen/split_sentences.json?dl=1",
            "srl_res": "https://www.dropbox.com/s/54lloy84ka8mycp/srl_res.json?dl=1"
        }
    },
    "tweets_candidates_french_elections": 
    {
        "description": "Tweets of candidates at the French presidential elections (2022)",
        "language": "french",
        "srl_model": "",
        "links": 
        {
            "raw": "https://www.dropbox.com/s/qqlq8xn9x645f79/tweets_candidates_french_elections.csv?dl=1"
        }
    }
}



In [3]:
from relatio import load_data
df = load_data(dataset = "tweets_candidates_french_elections", content = "raw")
df = df[df['candidate'] == 'yjadot']
df.head()

,id,doc,date,candidate
29238,29237,"Hier, nous étions à #Rennes, place Hoche, pour...",2022-02-09T16:01:00.000Z,yjadot
29239,29238,Pensées à ses proches,2022-02-09T15:31:36.000Z,yjadot
29240,29239,"Un an déjà que Guillaume, militant communiste ...",2022-02-09T15:31:16.000Z,yjadot
29241,29240,Le #OneOceanSummit s'ouvre aujourd'hui à #Bres...,2022-02-09T15:03:00.000Z,yjadot
29242,29241,Pour revoir l'intégralité de mon passage sur @...,2022-02-09T14:00:03.000Z,yjadot


In [4]:
from relatio import Preprocessor

import string
alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string) + ['rt']

p = Preprocessor(
    spacy_model = "fr_core_news_sm",
    remove_punctuation = True,
    remove_digits = True,
    lowercase = True,
    lemmatize = True,
    remove_chars = ["\"",'-',"^",".","?","!",";","(",")",",",":","\'","+","&","|","/","{","}",
                    "~","_","`","[","]",">","<","=","*","%","$","@","#","’"],
    stop_words = alphabet_list,
    n_process = -1,
    batch_size = 100
)

doc_index, sentences = p.split_into_sentences(
    df, output_path = None, progress_bar = True
)

Splitting into sentences...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3432/3432 [00:03<00:00, 978.78it/s]


In [5]:
sentence_index, roles = p.extract_svos(sentences, progress_bar = True)

for svo in roles[0:5]: print(svo)

Extracting SVOs...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7661/7661 [00:04<00:00, 1619.38it/s]

{'ARG0': 'nous', 'B-ARGM-NEG': False, 'B-V': 'étions', 'ARG1': ''}
{'ARG0': 'nous', 'B-ARGM-NEG': False, 'B-V': 'serons', 'ARG1': ''}
{'ARG0': 'vous', 'B-ARGM-NEG': True, 'B-V': 'pouvez', 'ARG1': ''}
{'ARG0': 'vous', 'B-ARGM-NEG': False, 'B-V': 'voici', 'ARG1': ''}
{'ARG0': 'vous', 'B-ARGM-NEG': False, 'B-V': 'suivre', 'ARG1': ''}


In [6]:
postproc_roles = p.process_roles(roles, 
                                 dict_of_pos_tags_to_keep = {
                                     "ARG0": ['PRON', 'NOUN', 'PROPN'],
                                     "B-V": ['VERB'],
                                     "ARG1": ['NOUN', 'PROPN', 'PRON']
                                 }, 
                                 max_length = 50,
                                 progress_bar = True,
                                 output_path = 'postproc_roles.json')

from relatio.utils import load_roles
postproc_roles = load_roles('postproc_roles.json')

for d in postproc_roles[0:5]: print(d)

Cleaning phrases for role ARG0...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10054/10054 [00:04<00:00, 2030.54it/s]


Cleaning phrases for role B-V...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10054/10054 [00:04<00:00, 2071.37it/s]


Cleaning phrases for role B-ARGM-MOD...


...

Cleaning phrases for role ARG1...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10054/10054 [00:04<00:00, 2076.95it/s]


Cleaning phrases for role ARG2...


0it [00:00, ?it/s]


{'ARG0': 'nous', 'B-ARGM-NEG': False}
{'ARG0': 'nous', 'B-ARGM-NEG': False}
{'ARG0': 'vous', 'B-V': 'pouvoir', 'B-ARGM-NEG': True}
{'ARG0': 'vous', 'B-V': 'voici', 'B-ARGM-NEG': False}
{'ARG0': 'vous', 'B-V': 'suivre', 'B-ARGM-NEG': False}


In [7]:
known_entities = p.mine_entities(
    sentences, 
    clean_entities = True, 
    progress_bar = True,
    output_path = 'entities.pkl'
)

from relatio.utils import load_entities
known_entities = load_entities('entities.pkl')

for n in known_entities.most_common(10): print(n)

Mining named entities...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7661/7661 [00:09<00:00, 849.06it/s]

('', 521)
('écologie', 92)
('parlement européen', 27)
('écologi', 21)
('primaireecologist', 19)
('ue', 17)
('union européen', 14)
('humanité', 13)
('avenir', 13)
('tva', 11)


In [8]:
top_known_entities = [e[0] for e in list(known_entities.most_common(100)) if e[0] != '']

In [9]:
from relatio import Embeddings
nlp_model = Embeddings("spaCy", "fr_core_news_sm", sentences=sentences) 

In [10]:
from relatio import NarrativeModel
from relatio.utils import prettify
from collections import Counter

In [11]:
m1 = NarrativeModel(model_type = 'hdbscan',
                   roles_considered = ['ARG0', 'B-V', 'B-ARGM-NEG', 'ARG1'],
                   roles_with_known_entities = ['ARG0','ARG1'],
                   known_entities = top_known_entities,
                   assignment_to_known_entities = 'character_matching',
                   roles_with_unknown_entities = ['ARG0','ARG1'],
                   embeddings_model = nlp_model,
                   threshold = 1)    

m1.fit(postproc_roles)

Embedding phrases...
Computing phrase embeddings...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2928/2928 [00:11<00:00, 247.28it/s]


Clustering phrases into clusters...
Labeling the clusters by the most frequent phrases...


In [12]:
narratives = m1.predict(postproc_roles, progress_bar = True)


Predicting entities for role: ARG0...
Matching known entities (with character matching)...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5657/5657 [00:00<00:00, 15081.23it/s]


Matching unknown entities (with clustering model)...
Computing phrase embeddings...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5657/5657 [00:15<00:00, 372.80it/s]


Dimension reduction of vectors (PCA + UMAP)...
Assignment to clusters...
Assigning labels to matches...

Predicting entities for role: ARG1...
Matching known entities (with character matching)...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3556/3556 [00:00<00:00, 13900.53it/s]


Matching unknown entities (with clustering model)...
Computing phrase embeddings...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3556/3556 [00:11<00:00, 299.26it/s]


Dimension reduction of vectors (PCA + UMAP)...
Assignment to clusters...
Assigning labels to matches...


In [13]:
pretty_narratives = []
for n in narratives: 
    if n.get('ARG0') is not None:
        if n.get('B-V') is not None:
            if n.get('ARG1') is not None:
                pretty_narratives.append(prettify(n))
                
pretty_narratives = Counter(pretty_narratives)
for t in pretty_narratives.most_common(10): print(t)

('nous conditionner euro argent', 7)
('nous donne millier milliard euro', 7)
('nous gagner alternative qui chance', 4)
('nous gagner élection', 4)
('nous reprendre meeting', 3)
('nous créer emploi', 3)
('énergie créer emploi', 3)
('nous donner moyen', 2)
('nous propose primaire', 2)
('nous allouer euro argent', 2)


In [14]:
m2 = NarrativeModel(model_type = 'kmeans',
                   roles_considered = ['ARG0', 'B-V', 'B-ARGM-NEG', 'ARG1'],
                   roles_with_known_entities = ['ARG0','ARG1'],
                   known_entities = top_known_entities,
                   assignment_to_known_entities = 'character_matching',
                   roles_with_unknown_entities = ['ARG0','ARG1'],
                   embeddings_model = nlp_model,
                   threshold = 0.3)    

m2.fit(postproc_roles, progress_bar = True)

Embedding phrases...
Computing phrase embeddings...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2928/2928 [00:11<00:00, 250.88it/s]


Clustering phrases into clusters...
Labeling the clusters by the most frequent phrases...


In [15]:
narratives = m2.predict(postproc_roles, progress_bar = True)


Predicting entities for role: ARG0...
Matching known entities (with character matching)...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5657/5657 [00:00<00:00, 15104.49it/s]


Matching unknown entities (with clustering model)...
Computing phrase embeddings...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5657/5657 [00:15<00:00, 361.17it/s]


Dimension reduction of vectors (PCA + UMAP)...
Assignment to clusters...
Assigning labels to matches...

Predicting entities for role: ARG1...
Matching known entities (with character matching)...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3556/3556 [00:00<00:00, 13545.46it/s]


Matching unknown entities (with clustering model)...
Computing phrase embeddings...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3556/3556 [00:11<00:00, 301.50it/s]


Dimension reduction of vectors (PCA + UMAP)...
Assignment to clusters...
Assigning labels to matches...


In [16]:
pretty_narratives = []
for n in narratives: 
    if n.get('ARG0') is not None:
        if n.get('B-V') is not None:
            if n.get('ARG1') is not None:
                pretty_narratives.append(prettify(n))
                
pretty_narratives = Counter(pretty_narratives)
for t in pretty_narratives.most_common(10): print(t)

('nous conditionner euro argent', 7)
('nous donne millier milliard euro', 7)
('nous gagner élection', 4)
('nous reprendre andré etienne', 3)
('nous gagner présidentielle', 3)
('nous créer emploi', 3)
('nous redonner sens', 3)
('énergie créer emploi', 3)
('nous investir milliard euro', 2)
('nous mettre sens', 2)


In [17]:
from relatio import build_graph, draw_graph

G = build_graph(
    narratives, 
    top_n = 100, 
    prune_network = True
)

draw_graph(
    G,
    notebook = False,
    show_buttons = False,
    width="1600px",
    height="1000px",
    output_filename = 'example.html'
    )